In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from functools import partial
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
import features
import warnings
warnings.filterwarnings("ignore")

In [3]:
path = '/Users/ewenwang/Documents/loan'
filename = 'train_v2.csv'
st = datetime.now()
dataset = pd.read_csv(os.path.join(path, filename), low_memory=False)
dataset = pd.DataFrame(dataset)
print('\nActually, ', datetime.now()-st)

dataset.shape


Actually,  0:00:40.876253


(105471, 771)

In [3]:
dataset.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,...,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,...,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,...,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,...,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,...,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 771 entries, id to loss
dtypes: float64(652), int64(99), object(19), uint64(1)
memory usage: 620.4+ MB


In [5]:
np.where(dataset.dtypes == 'uint64')

(array([462]),)

In [40]:
# unitFeature = dataset.columns[462]

In [1]:
# dataset[unitFeature]

In [57]:
dataset.iloc[0:4, :9]

,id,f1,f2,f3,f4,f5,f6,f7,f8
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0


In [52]:
np.where(dataset.head(5000).T.duplicated())[0]

array([ 32,  33,  35,  36,  56,  84,  85,  86,  94,  95,  96, 104, 105,
       106, 114, 115, 116, 124, 125, 126, 153, 154, 155, 163, 164, 165,
       173, 174, 175, 183, 184, 185, 193, 194, 195, 223, 224, 225, 233,
       234, 235, 243, 244, 245, 253, 254, 255, 263, 264, 265, 292, 293,
       294, 300, 301, 302, 308, 309, 310, 316, 317, 318, 324, 325, 326,
       343, 352, 360, 369, 377, 406, 415, 425, 455, 472, 482, 492, 502,
       547, 557, 567, 576, 593, 669, 691, 692, 693, 720, 732, 755, 761])

In [36]:
dataset.head(5000).T.duplicated().any()

True

In [4]:
df = dataset

In [97]:
def features_dup(df, n_head = 5000, print_dup = False):
    """Check first n_head rows and obtain duplicated features."""
    if dataset.head(n_head).T.duplicated().any():
        dup_list = np.where(df.head(n_head).T.duplicated())[0].tolist()
        dup_features = df.columns[dup_list]
        if print_dup:
            print(dup_features)
    return dup_features.tolist()

In [109]:
d = features_dup(df)

In [111]:
# d.tolist()

In [124]:
# [target, 'id', d.tolist()]
# d.tolist()+['id', target]

In [110]:
len(d)

92

In [105]:
target = 'loss'
features = [x for x in dataset.columns if x not in ['id', 'defualt', target, d]]

In [108]:
# features

In [107]:
len(features)

770

In [8]:
def features_clf(df, features):
    dtypes = df[features].dtypes.apply(lambda x: x.name).to_dict()
    int_features, float_features, object_features = [], [], []
    for col, dtype in dtypes.items():
        if dtype == 'int64':
            int_features.append(col)
        elif dtype == 'float64':
            float_features.append(col)
        elif dtype == 'object':
            object_features.append(col)
    return int_features, float_features, object_features

In [26]:
# dtypes = df[features].dtypes.apply(lambda x: x.name).to_dict()

In [19]:
int_cols, float_cols, object_cols = [], [], []
for col, dtype in dtypes.items():
    if dtype == 'object':
        object_cols.append(col)
    elif dtype == 'float64':
        float_cols.append(col)
    elif dtype == 'int64':
        int_cols.append(col)

In [17]:
len(int_cols)+len(float_cols)+len(object_cols)+2

771

In [9]:
i, f, o = features_clf(df, features)

In [14]:
len(d)

91

In [15]:
len(i) + len(f) + len(o)

677

In [51]:
# find_corr_pairs_sub(df[f])

In [10]:
def corr_pairs(df, gamma=0.99999):
    corr_matrix = df.corr().abs()
    os = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
          .stack()
          .sort_values(ascending=False))
    return os[os>gamma].index.values.tolist()

In [11]:
i_l = corr_pairs(df[i])

In [18]:
i_l

[('f74', 'f722'), ('f36', 'f74'), ('f36', 'f722')]

In [12]:
f_l = corr_pairs(df[f])

In [84]:
f_l

[('f676', 'f765'),
 ('f21', 'f763'),
 ('f7', 'f439'),
 ('f264', 'f578'),
 ('f115', 'f484'),
 ('f254', 'f569'),
 ('f85', 'f453'),
 ('f95', 'f494'),
 ('f224', 'f539'),
 ('f105', 'f504'),
 ('f234', 'f549'),
 ('f244', 'f559'),
 ('f8', 'f543'),
 ('f577', 'f578'),
 ('f483', 'f484'),
 ('f264', 'f577'),
 ('f568', 'f569'),
 ('f115', 'f483'),
 ('f452', 'f453'),
 ('f254', 'f568'),
 ('f493', 'f494'),
 ('f85', 'f452'),
 ('f538', 'f539'),
 ('f406', 'f768'),
 ('f503', 'f504'),
 ('f548', 'f549'),
 ('f95', 'f493'),
 ('f224', 'f538'),
 ('f558', 'f559'),
 ('f105', 'f503'),
 ('f234', 'f548'),
 ('f527', 'f528'),
 ('f244', 'f558'),
 ('f274', 'f527'),
 ('f274', 'f528'),
 ('f42', 'f644'),
 ('f389', 'f625'),
 ('f7', 'f608'),
 ('f439', 'f608')]

In [85]:
n_f_l = len(f_l)
n_f_l

39

In [43]:
df[df.columns & i_l[0]].head()

,f74,f722
0,5,6
1,6,7
2,13,14
3,4,5
4,26,27


In [46]:
abs(df[i_l[0][0]]-df[i_l[0][1]]).unique()

array([1])

In [63]:
for index, value in enumerate(i_l):
    print(index, '-'.join(value))

0 f74-f722
1 f36-f74
2 f36-f722


In [64]:
new_i = pd.DataFrame()
for index, value in enumerate(i_l):
    new_i['-'.join(value)] = abs(df[i_l[index][0]]-df[i_l[index][1]])

In [65]:
new_i.head()

,f74-f722,f36-f74,f36-f722
0,1,0,1
1,1,0,1
2,1,0,1
3,1,0,1
4,1,0,1


In [69]:
new_i.describe()

,f74-f722,f36-f74,f36-f722
count,105471.0,105471.000000,105471.000000
mean,1.0,0.001489,1.001489
std,0.0,0.038553,0.038553
min,1.0,0.000000,1.000000
25%,1.0,0.000000,1.000000
50%,1.0,0.000000,1.000000
75%,1.0,0.000000,1.000000
max,1.0,1.000000,2.000000


In [72]:
for i in new_i.columns:
    print(new_i[i].unique())

[1]
[0 1]
[1 2]


In [81]:
len((df[i_l[0][0]]-df[i_l[0][1]]).unique())

1

In [13]:
def features_new(df, ls):
    new = pd.DataFrame()
    for index, value in enumerate(ls):
        if len((df[ls[index][0]]-df[ls[index][1]]).unique()) > 1:
            new['-'.join(value)] = df[ls[index][0]]-df[ls[index][1]]
    return new   

In [80]:
features_new(df, i_l).head()

,f36-f74,f36-f722
0,0,-1
1,0,-1
2,0,-1
3,0,-1
4,0,-1


In [15]:
new_i = features_new(df, i_l)

In [16]:
new_f = features_new(df, f_l)

In [88]:
new_f.describe()

,f676-f765,f21-f763,f7-f439,f264-f578,f115-f484,f254-f569,f85-f453,f95-f494,f224-f539,f105-f504,...,f105-f503,f234-f548,f527-f528,f244-f558,f274-f527,f274-f528,f42-f644,f389-f625,f7-f608,f439-f608
count,105470.0,1.036310e+05,105289.000000,105297.000000,105136.000000,105297.000000,105136.000000,105136.000000,105297.000000,105136.000000,...,105136.000000,105297.000000,105370.000000,105297.000000,105370.000000,105370.000000,105469.000000,104773.000000,105270.000000,105270.000000
mean,1.0,1.000000e+00,0.000038,0.031978,0.021674,0.021271,0.010875,0.010799,0.010637,0.010751,...,0.095420,0.094411,-1044.903701,0.094870,-2.062121,-1046.965822,0.002095,0.002157,6.851449,6.851411
std,0.0,1.035788e-17,0.006164,0.049767,0.040132,0.042264,0.030071,0.029932,0.030983,0.029867,...,0.172139,0.175715,4345.411083,0.175305,6226.262833,7636.026910,0.055813,0.056757,11.445368,11.445392
min,1.0,1.000000e+00,0.000000,-0.300000,0.000000,-0.200000,0.000000,0.000000,-0.100000,0.000000,...,0.000000,-0.560000,-364241.990000,-0.570000,-711219.040000,-877109.430000,0.000000,0.000000,0.000000,-1.000000
25%,1.0,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-906.742500,0.000000,-89.967500,-544.107500,0.000000,0.000000,0.000000,0.000000
50%,1.0,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-313.625000,0.000000,58.725000,0.000000,0.000000,0.000000,2.000000,2.000000
75%,1.0,1.000000e+00,0.000000,0.100000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.120000,0.120000,-85.692500,0.120000,309.292500,1.980000,0.000000,0.000000,8.000000,8.000000
max,1.0,1.000000e+00,1.000000,0.410000,0.110000,0.310000,0.110000,0.110000,0.210000,0.110000,...,0.980000,1.280000,0.000000,1.250000,609823.870000,379757.440000,3.000000,3.000000,72.000000,72.000000


In [17]:
new_f['f527-f528'].head()

0   -414.28
1    -35.73
2   -122.26
3   -120.58
4    -21.47
Name: f527-f528, dtype: float64

In [19]:
from sklearn.linear_model import LogisticRegression as LR

In [39]:
pd.DataFrame(new_f['f527-f528'].fillna(new_f['f527-f528'].median())).head()

,f527-f528
0,-414.28
1,-35.73
2,-122.26
3,-120.58
4,-21.47


In [40]:
lr = LR()
lr.fit(pd.DataFrame(new_f['f527-f528'].fillna(new_f['f527-f528'].median())), df[target])
# new_f['f527-f528'].fillna(new_f['f527-f528'].median()).reshape(-1, 1)[:10,:]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
prob = lr.predict_proba(pd.DataFrame(new_f['f527-f528'].fillna(new_f['f527-f528'].median())))

In [43]:
prob.dtype

dtype('float64')

In [45]:
df[target].unique()

array([  0,   1,  16,  19,   4,  11,  21,   2,   9,   5,   3,   7,  13,
         6,  25,  44,  17, 100,  10,   8,  65,  15,  12,  29,  22,  20,
        18,  41,  55,  28,  23,  14,  38,  42,  33,  27,  24,  39,  26,
        31,  35,  64,  32,  54,  37,  61,  79,  51,  43,  77,  47,  36,
        46,  81,  78,  60,  45,  85,  84,  34,  90,  40,  50,  92,  74,
        62,  53,  59,  89,  48,  30,  49,  99,  73,  95,  83,  56,  70,
        71,  68,  94,  72,  57,  58,  52,  63,  69,  76,  80])

In [50]:
default = df[target]
default[default>0] = 1

In [60]:
df['default'] = default

In [51]:
gf = pd.DataFrame(new_f['f527-f528'].fillna(new_f['f527-f528'].median()))

In [52]:
lr = LR()
lr.fit(gf, default)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [58]:
prob = lr.predict_proba(gf)[:,1]

In [57]:
prob[:10,:]

array([[ 0.91826533,  0.08173467],
       [ 0.92285023,  0.07714977],
       [ 0.92182344,  0.07817656],
       [ 0.92184349,  0.07815651],
       [ 0.92301825,  0.07698175],
       [ 0.91895742,  0.08104258],
       [ 0.92236336,  0.07763664],
       [ 0.90897127,  0.09102873],
       [ 0.91189284,  0.08810716],
       [ 0.91951075,  0.08048925]])

In [59]:
print("AUC Score (Train): %f" % metrics.roc_auc_score(default, prob))

AUC Score (Train): 0.915607


In [66]:
temp = pd.DataFrame(df[i_l[0][0]]-df[i_l[0][1]])

In [81]:
temp.values.ravel().unique()

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [95]:
def features_new(df, ls, target, auc_score=0.75, silent = False):
    new = pd.DataFrame()
    for index, value in enumerate(ls):
        temp = df[ls[index][0]]-df[ls[index][1]]
        if len(temp.unique()) > 1:
            temp = pd.DataFrame(temp.fillna(temp.median()))
            lr = LR()
            lr.fit(temp, df[target])
            prob = lr.predict_proba(temp)[:,1]
            auc = metrics.roc_auc_score(df[target], prob)
            if auc > auc_score:
                if silent:
                    print('-'.join(value), ' AUC (train): ', auc)
                new['-'.join(value)] = df[ls[index][0]]-df[ls[index][1]]
    return new

In [91]:
features_new(df, i_l, 'default').head()

""


In [96]:
features_new(df, f_l, target='default', auc_score=0.75, silent=True).head()

f527-f528  AUC (train):  0.915607274654
f274-f527  AUC (train):  0.900140457424
f274-f528  AUC (train):  0.937543347206


,f527-f528,f274-f527,f274-f528
0,-414.28,-105.69,-519.97
1,-35.73,35.73,0.00
2,-122.26,-137.84,-260.10
3,-120.58,-8.78,-129.36
4,-21.47,21.50,0.03


In [ ]:
##LDA
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

cl = data_x.columns
while (len(cl) - 20 > 5):
    lda = LDA(n_components=2)
    all_features=StandardScaler().fit_transform(data_x[cl])
    X_r2 = lda.fit(all_features, train_loss).transform(all_features)
    result_coff=pd.DataFrame({'title':data_x[cl].columns,'coff':lda.coef_.reshape(data_x[cl].shape[1])})
    result_coff['coff']=np.abs(result_coff['coff'])
    cl = list(result_coff.sort_values(ascending=False,by=['coff']).iloc[:,1].values)
    del cl[-20:]
    print('Turns Left:', len(cl)/20)

cl1 = list(itertools.combinations(cl, 3))
aucList = pd.DataFrame(columns= [['AUC values','features']]).astype('object')

for i in range(len(cl1)):
        golden_features = data_x[list(cl1[i])]
        lda = LDA(n_components=2)
        X_r2 = lda.fit(golden_features, train_loss).transform(golden_features)
        aucList.loc[i] = [dq.F_AUC(X_r2,train_loss,features=['lda']).loc[0][0], list(cl1[i])]
        print('Turns Remaining:', len(cl1)-i)

aucList.sort_values(['AUC values'],ascending=[False])

In [28]:
corr_matrix = df[i].corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
os_i = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                 .stack()
                 .sort_values(ascending=False))
#first element of os series is the pair with the bigest correlation

In [34]:
pairs_i = os_i[os_i > .99999]

In [39]:
pairs_i

f74  f722    1.000000
f36  f74     0.999998
     f722    0.999998
dtype: float64

In [30]:
corr_matrix = df[f].corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
os_f = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                 .stack()
                 .sort_values(ascending=False))
#first element of os series is the pair with the bigest correlation

In [ ]:
c = df[i].corr().abs()
s = c.unstack()
so = s.order(kind="quicksort")

In [130]:
c = df[i].corr().abs()

In [148]:
# c.columns() 
np.where(c.isnull().all())

(array([ 9, 57]),)

In [143]:
c.head()

,f1,f2,f4,f5,f6,f13,f16,f24,f25,f33,...,f722,f725,f728,f730,f731,f732,f736,f776,f777,f778
f1,1.000000,0.210388,0.782389,0.269849,0.102250,0.056130,0.075052,0.299080,0.302254,NaN,...,0.081430,0.080732,0.003761,0.083830,0.176596,0.267671,NaN,0.078637,0.873927,0.270134
f2,0.210388,1.000000,0.215980,0.221009,0.091765,0.063015,0.260137,0.026845,0.494467,NaN,...,0.078268,0.241238,0.051839,0.021537,0.011294,0.044443,NaN,0.097982,0.225693,0.221553
f4,0.782389,0.215980,1.000000,0.256425,0.119798,0.049142,0.106806,0.337085,0.216014,NaN,...,0.108048,0.088469,0.027852,0.063114,0.159975,0.255563,NaN,0.038133,0.875710,0.262051
f5,0.269849,0.221009,0.256425,1.000000,0.055828,0.113924,0.007983,0.097416,0.115824,NaN,...,0.084133,0.103954,0.057205,0.025927,0.008229,0.042850,NaN,0.036634,0.303485,0.770800
f6,0.102250,0.091765,0.119798,0.055828,1.000000,0.001726,0.230357,0.090395,0.057398,NaN,...,0.006449,0.010516,0.013691,0.021994,0.030764,0.039208,NaN,0.010537,0.161654,0.070763


In [132]:
s = c.unstack()

In [152]:
so = s.sort_values(kind="quicksort", ascending=False)

In [164]:
so[so > .99][so < 1]

f74   f36     0.999998
f36   f74     0.999998
      f722    0.999998
f722  f36     0.999998
f681  f74     0.999855
f74   f681    0.999855
f681  f722    0.999855
f722  f681    0.999855
f36   f681    0.999854
f681  f36     0.999854
f467  f447    0.999364
f447  f467    0.999364
f532  f521    0.998688
f521  f532    0.998688
f728  f681    0.994136
f681  f728    0.994136
f728  f36     0.993618
f36   f728    0.993618
f722  f728    0.993612
f728  f74     0.993612
f74   f728    0.993612
f728  f722    0.993612
      f730    0.991874
f730  f728    0.991874
      f731    0.990528
f731  f730    0.990528
      f732    0.990072
f732  f731    0.990072
dtype: float64

In [90]:
from scipy.stats.stats import pearsonr
pearsonr(dataset.f527, dataset.f528)

(nan, 1.0)

In [5]:
dataset.f527.isnull().any()

True

In [6]:
df = dataset

In [11]:
df.f527 = df.f527.fillna(df.f527.median())
df.f528 = df.f528.fillna(df.f528.median())
# np.corrcoef(df.f527, df.f528)

In [13]:
np.corrcoef(df.f527, df.f528)[0, 1]

0.99999977947009588

In [15]:
target = df.loss
target[target > 0] = 1

In [16]:
np.corrcoef(df.f527-df.f528, target)[0, 1]

-0.18959896011181998

In [95]:
#dataset.f527

In [50]:
df = dataset
n_head = 250
#print(np.where(dataset.head(n_head).T.duplicated())[0])
df.iloc[0:10, np.where(dataset.head(n_head).T.duplicated())[0]]

,f34,f35,f37,f38,f58,f86,f87,f88,f96,f97,...,f678,f700,f701,f702,f724,f725,f729,f741,f764,f770
0,0,0,0,0,5.0,862.0,862.0,862.0,851.0,851.0,...,0.0,0,0,0,0.0,0,6,6,1,5
1,0,0,0,0,6.0,26.0,26.0,26.0,20.0,20.0,...,0.0,0,0,0,0.0,0,7,7,1,6
2,0,0,0,0,13.0,134.0,134.0,134.0,124.0,124.0,...,0.0,0,0,0,0.0,0,14,14,1,13
3,0,0,0,0,4.0,1088.0,1088.0,1088.0,903.0,903.0,...,0.0,0,0,0,0.0,0,5,5,1,4
4,0,0,0,0,26.0,5.0,5.0,5.0,5.0,5.0,...,0.0,0,0,0,0.0,0,27,27,1,26
5,0,0,0,0,22.0,137.0,137.0,137.0,131.0,131.0,...,0.0,0,0,0,0.0,0,23,23,1,22
6,0,0,0,0,23.0,349.0,349.0,349.0,399.0,399.0,...,0.0,0,0,0,0.0,0,24,24,1,23
7,0,0,0,0,17.0,956.0,956.0,956.0,837.0,837.0,...,0.0,0,0,0,0.0,0,18,18,1,17
8,0,0,0,0,7.0,239.0,239.0,239.0,253.0,253.0,...,0.0,0,0,0,0.0,0,8,8,1,7
9,0,0,0,0,5.0,83.0,83.0,83.0,82.0,82.0,...,0.0,0,0,0,0.0,0,6,6,1,5
